In [296]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

,date,F1,F2,F1 Running Total,F2 Running Total
0,2020-12-1,10,5,10,5
1,2020-12-3,9,6,19,11
2,2020-12-4,7,16,26,27


In [188]:
d="""Date/time,Flag,Type,cuml_sell_qty,cuml_buy_qty,expected col
2021-03-05 13:00:00,1,BOD,0.000,0.000 
2021-03-05 13:00:52,2,action,-0.008,0.000,-0.008
2021-03-05 13:00:53,2,action,-0.545,0.000,-0.545
2021-03-06 01:00:55,2,action,-0.545,0.019,-0.526
2021-03-06 13:00:00,3,BOD,,,-0.526
2021-03-06 13:00:54,4,action,,0.176,-0.350
2021-03-07 01:01:08,4,action,,0.204,-0.322
2021-03-07 13:00:00,5,BOD,,0.204,-0.147
2021-03-08 01:06:04,6,action,,0.013,-0.309
2021-03-08 13:00:00,7,BOD,,,-0.309
2021-03-08 13:01:02,8,action,,0.309,0.000
2021-03-09 01:00:55,8,action,-0.296,0.309,-0.296"""
df=g(d)
df

,Date/time,Flag,Type,cuml_sell_qty,cuml_buy_qty,expected col
0,2021-03-05 13:00:00,1,BOD,0.000,0.000,NaN
1,2021-03-05 13:00:52,2,action,-0.008,0.000,-0.008
2,2021-03-05 13:00:53,2,action,-0.545,0.000,-0.545
3,2021-03-06 01:00:55,2,action,-0.545,0.019,-0.526
4,2021-03-06 13:00:00,3,BOD,NaN,NaN,-0.526
5,2021-03-06 13:00:54,4,action,NaN,0.176,-0.350
6,2021-03-07 01:01:08,4,action,NaN,0.204,-0.322
7,2021-03-07 13:00:00,5,BOD,NaN,0.204,-0.147
8,2021-03-08 01:06:04,6,action,NaN,0.013,-0.309
9,2021-03-08 13:00:00,7,BOD,NaN,NaN,-0.309


In [215]:
weekmask_egypt = "Sun Mon Tue Wed Thu"
# They also observe International Workers' Day so let's # add that for a couple of years
holidays=["2018-05-01",datetime.datetime(2019,5,1),
          np.datetime64("2020-05-01"),]
bday_egypt = pd.offsets.CustomBusinessDay(
    holidays = holidays,
    weekmask = weekmask_egypt
    )
dt = datetime.datetime(2021,4,30)
dt + 2 * bday_egypt # Timestamp('2021-05-03 00:00:00')

bday_egypt.apply(dt) # Timestamp('2021-05-02 00:00:00')

Timestamp('2021-05-02 00:00:00')

In [226]:
dts = pd.date_range(dt, periods=5, freq=bday_egypt)
pd.Series(dts.weekday, dts).map(pd.Series("Mon Tue Wed Thu Fri Sat Sun".split()))

2021-05-02    Sun
2021-05-03    Mon
2021-05-04    Tue
2021-05-05    Wed
2021-05-06    Thu
Freq: C, dtype: object

In [228]:
import pandas.tseries.holiday
print(pandas.tseries.holiday.USFederalHolidayCalendar)

<class 'pandas.tseries.holiday.USFederalHolidayCalendar'>


In [258]:
df=pd.DataFrame({'month':['1','1','1','1','1','2','2','2','2','2','2','2'],'X1': 
[30,42,25,32,12,10,4,6,5,10,24,21],'X2':[10,76,100,23,65,94,67,24,67,54,87,81],'X3': 
[23,78,95,52,60,76,68,92,34,76,34,12]})  
df

,month,X1,X2,X3
0,1,30,10,23
1,1,42,76,78
2,1,25,100,95
3,1,32,23,52
4,1,12,65,60
5,2,10,94,76
6,2,4,67,68
7,2,6,24,92
8,2,5,67,34
9,2,10,54,76


In [262]:
from sklearn.cluster import KMeans
cols = df.columns[2:4]
mapping = {0: 'weak', 1: 'average', 2: 'best'}


def cluster(X):
    k_means = KMeans(n_clusters=3).fit(X)
    return X.groupby(k_means.labels_)\
        .transform('mean').sum(1)\
        .rank(method='dense').sub(1)\
        .astype(int).to_frame()

df['Cluster_id'] = df.groupby('month')[cols].apply(cluster)
df['Cluster_cat'] = df['Cluster_id'].map(mapping)

ModuleNotFoundError: No module named 'sklearn'

In [7]:
d="""a,b
20,30
40,50"""
df=pd.read_csv(StringIO(d))
df

,a,b
0,20,30
1,40,50


In [22]:
d="""DEAL ID,DATE
432,20200405
522,20180405
632,20180409
9332,20190405"""
df=g(d)
df

,DEAL ID,DATE
0,432,20200405
1,522,20180405
2,632,20180409
3,9332,20190405


In [28]:
new_df = df.groupby(pd.to_datetime(df.DATE, format='%Y%m%d').dt.year).agg({'DEAL ID' : 'count'})
new_df

,DEAL ID
DATE,
2018,2
2019,1
2020,1


In [3]:
d="""C,A,V,D
9,apar,1.0,0
8,bpar,4.0,8
7,cpar,7.0,7
0,apar,8.0,6
8,apar,9.0,4
9,bpar,3.0,2"""
df=g(d)
df

,C,A,V,D
0,9,apar,1.0,0
1,8,bpar,4.0,8
2,7,cpar,7.0,7
3,0,apar,8.0,6
4,8,apar,9.0,4
5,9,bpar,3.0,2


In [13]:
pd.concat(df, np.where(df.A.eq('apar'), [df.A+'_t',df.V-.5], np.nan)

array([['apar_t', nan, nan, 'apar_t', 'apar_t', nan],
       [0.5, nan, nan, 7.5, 8.5, nan]], dtype=object)

In [15]:
pd.DataFrame(np.where(df.A.eq('apar'), [df.A+'_t',df.V-.5], np.nan)).T

,0,1
0,apar_t,0.5
1,NaN,NaN
2,NaN,NaN
3,apar_t,7.5
4,apar_t,8.5
5,NaN,NaN


In [25]:
df=pd.DataFrame({
    'Month':['2020-11', '2020-11', '2020-11', '2020-12', '2020-12', '2020-12',
            '2021-01','2021-01','2021-02','2021-02','2021-02','2021-02'],
    'ID':['A','B','B','A','A','C','A','A','C','A','A','B'],
    'f_count':[4,2,3,1,5,3,2,1,1,1,2,3],
    't_count':[5,4,1,0,8,4,1,0,2,3,5,7,]
})

df['Month'] = pd.to_datetime(df.Month)
df = df.set_index('Month')

df1 = df.last('M').groupby('ID').sum().reset_index().rename(
    columns={'f_count':'f_count(current month)',
            't_count':'t_count(current month)'})
df2 = df.last('3M').first('2M').groupby('ID').sum().reset_index().rename(
    columns={'f_count':'f_count(past 3 months)',
            't_count':'t_count(past 3 months)'})

df  = pd.merge(df1, df2, on='ID', how='inner').reindex(columns = [ 'ID',
    'f_count(current month)', 'f_count(past 3 months)',
    't_count(current month)','t_count(past 3 months)'
])
df

,ID,f_count(current month),f_count(past 3 months),t_count(current month),t_count(past 3 months)
0,A,3,9,8,9
1,C,1,3,2,4


In [34]:
d="""Timestamp,Event,SID,User
2021-04-15 10:35,OPEN,7933711,user2
2021-04-15 10:37,OPEN,8022822,user1
2021-04-15 10:37,OPEN,7933711,user2
2021-04-15 10:41,OPEN,1234567,user2
2021-04-15 10:45,OPEN,1234567,user2
2021-04-15 11:39,OPEN,8022822,user1
2021-04-15 11:59,DO,7933711,user2
2021-04-15 12:24,OPEN,7933711,user2
2021-04-15 12:59,DO,8022822,user1
2021-04-15 13:25,OPEN,8022822,user1
2021-04-15 13:29,OPEN,7933711,user2
2021-04-15 14:27,OPEN,8022822,user1"""
df=g(d)
df.tail()

,Timestamp,Event,SID,User
7,2021-04-15 12:24,OPEN,7933711,user2
8,2021-04-15 12:59,DO,8022822,user1
9,2021-04-15 13:25,OPEN,8022822,user1
10,2021-04-15 13:29,OPEN,7933711,user2
11,2021-04-15 14:27,OPEN,8022822,user1


In [35]:
df['Timestamp'] = pd.to_datetime(df.Timestamp)
#df = df.set_index('Timestamp')
df

,Timestamp,Event,SID,User
0,2021-04-15 10:35:00,OPEN,7933711,user2
1,2021-04-15 10:37:00,OPEN,8022822,user1
2,2021-04-15 10:37:00,OPEN,7933711,user2
3,2021-04-15 10:41:00,OPEN,1234567,user2
4,2021-04-15 10:45:00,OPEN,1234567,user2
5,2021-04-15 11:39:00,OPEN,8022822,user1
6,2021-04-15 11:59:00,DO,7933711,user2
7,2021-04-15 12:24:00,OPEN,7933711,user2
8,2021-04-15 12:59:00,DO,8022822,user1
9,2021-04-15 13:25:00,OPEN,8022822,user1


In [48]:
df.Timestamp = pd.to_datetime(df.Timestamp) # if not already datetime

df = df.sort_values(['User', 'Timestamp', 'SID', 'Event'])
print(df)
df.Event.eq('DO').shift()

             Timestamp Event      SID   User
1  2021-04-15 10:37:00  OPEN  8022822  user1
5  2021-04-15 11:39:00  OPEN  8022822  user1
8  2021-04-15 12:59:00    DO  8022822  user1
9  2021-04-15 13:25:00  OPEN  8022822  user1
11 2021-04-15 14:27:00  OPEN  8022822  user1
0  2021-04-15 10:35:00  OPEN  7933711  user2
2  2021-04-15 10:37:00  OPEN  7933711  user2
3  2021-04-15 10:41:00  OPEN  1234567  user2
4  2021-04-15 10:45:00  OPEN  1234567  user2
6  2021-04-15 11:59:00    DO  7933711  user2
7  2021-04-15 12:24:00  OPEN  7933711  user2
10 2021-04-15 13:29:00  OPEN  7933711  user2


1       NaN
5     False
8     False
9      True
11    False
0     False
2     False
3     False
4     False
6     False
7      True
10    False
Name: Event, dtype: object

In [49]:
df = pd.DataFrame({ 
    'BU': ['Total', 'Total', 'Total', 'CRS', 'CRS', 'CRS'], 
    'Line_Item': ['Revenues','EBT', 'Expenses', 'Revenues', 'EBT', 'Expenses'],
    '1Q16-1': [100, 120, 0, 200, 190, 210],
    '2Q16-1': [100, 0, 130, 200, 190, 210],
    '3Q16-1': [200, 250, 0, 120, 0, 190],
    '1Q16-2': [100, 120, 0, 200, 190, 210],
    '2Q16-2': [100, 0, 130, 200, 190, 210],
    '3Q16-2': [200, 250, 0, 120, 0, 190]
})
df

,BU,Line_Item,1Q16-1,2Q16-1,3Q16-1,1Q16-2,2Q16-2,3Q16-2
0,Total,Revenues,100,100,200,100,100,200
1,Total,EBT,120,0,250,120,0,250
2,Total,Expenses,0,130,0,0,130,0
3,CRS,Revenues,200,200,120,200,200,120
4,CRS,EBT,190,190,0,190,190,0
5,CRS,Expenses,210,210,190,210,210,190


In [55]:
df.filter(like=*[f'{for i in range(1,4)}Q16'])

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-55-052602e4f9b3>, line 1)

In [65]:
df.assign(**{
    f'{i}Q16': df.filter(like=f'{i}Q16').sum(1) for i in [1,2,3]
})


print(df.filter(like=f'{i}Q16').sum(1) for i in [1,2,3])


<generator object <genexpr> at 0x12d75bb30>


In [69]:
d3 = pd.DataFrame({
    'DIST_EVENT':[1,2,0,3],
    'DIST_NON_EVENT':[2,3,4,0]
})

In [72]:
np.where(d3.DIST_EVENT * d3.DIST_NON_EVENT == 0 ,np.nan ,np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT))

/Users/utsav/.conda/envs/myenv/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


array([-0.69314718, -0.40546511,         nan,         nan])

In [86]:
df= pd.DataFrame({
    'Country':['a','b','a','b'],
    'Height':[10,20,30,40]
})

df.groupby("Country", as_index=False).Height.mean().sort_values('Height', ascending=False)

,Country,Height
1,b,30
0,a,20


In [111]:
d="""yearmon,college,major,gpa,num
20140401,1,a,3.36,29
20180401,2,b,2.63,48
20160401,3,c,3.23,55
20170401,4,d,4.22,1
20140401,3,b,3.72,72"""
df=g(d)
df

,yearmon,college,major,gpa,num
0,20140401,1,a,3.36,29
1,20180401,2,b,2.63,48
2,20160401,3,c,3.23,55
3,20170401,4,d,4.22,1
4,20140401,3,b,3.72,72


In [112]:
df = df.groupby('major').apply(lambda x: np.average(x['gpa'], weights=x['num']))
df

major
a    3.360
b    3.284
c    3.230
d    4.220
dtype: float64

In [142]:
import pandas as pd

data = {'Company': ["0652369- INTER SUPPORT LLP, 202011","CIRCLE TRADING LTD 1-593616, 2020-06, 0201","Area  Food Service Co., Ltd.-6958047, 2020-07"]}

df = pd.DataFrame(data)

df['co'] = df['Company'].str.extract('(\d{5,})')
print (df['co'])

0    0652369
1     593616
2    6958047
Name: co, dtype: object


In [144]:
import pandas as pd

df = pd.DataFrame({'Labels': {0: 'Apple',  1: 'Apple',  2: 'Apple',  3: 'Apple',  4: 'Orange',  5: 'Orange',  6: 'Orange',  7: 'Orange',  8: 'Grapes',  9: 'Grapes'}, 'Pattern': {0: 'Red',  1: 'Red',  2: 'Green',  3: 'Green',  4: 'Good',  5: 'Good',  6: 'Bad',  7: 'Bad',  8: 'Violet',  9: 'Violet'}, 'Status': {0: 'Checked',  1: 'Not_Checked',  2: 'Checked',  3: 'Not_Checked',  4: 'Checked',  5: 'Not_Checked',  6: 'Checked',  7: 'Not_Checked',  8: 'Checked',  9: 'Not_Checked'}, 'Count': {0: 79,  1: 221,  2: 3,  3: 306,  4: 13,  5: 297,  6: 28,  7: 281,  8: 20,  9: 290}, 'Some_Link': {0: 'http://www.example.com/',  1: 'http://angle.example.com/',  2: 'https://example.com/',  3: 'http://www.example.com/www.php',  4: 'http://example.com/blow/bag',  5: 'https://www.example.com/?baby=brake',  6: 'https://www.example.org/?afternoon=approval&baseball=arithmetic',  7: 'https://example.net/babies/badge?amount=balance',  8: 'http://www.example.com/boundary/boat.aspx',  9: 'http://www.example.com/sssl.php'}, 'Some_Link2': {0: 'http://www.example.com/beef/approval',  1: 'https://www.example.com/qqa.php',  2: 'https://example.com/aswq.php',  3: 'http://www.example.com/believe/bike.php?amount=blade',  4: 'https://www.example.com/',  5: 'http://www.example.com/?beef=acoustics',  6: 'https://www.example.com/#apparatus',  7: 'https://www.example.com/asd.php',  8: 'http://basketball.example.com/bone/bedroom',  9: 'https://www.example.org/box/back'}})
df

,Labels,Pattern,Status,Count,Some_Link,Some_Link2
0,Apple,Red,Checked,79,http://www.example.com/,http://www.example.com/beef/approval
1,Apple,Red,Not_Checked,221,http://angle.example.com/,https://www.example.com/qqa.php
2,Apple,Green,Checked,3,https://example.com/,https://example.com/aswq.php
3,Apple,Green,Not_Checked,306,http://www.example.com/www.php,http://www.example.com/believe/bike.php?amount...
4,Orange,Good,Checked,13,http://example.com/blow/bag,https://www.example.com/
5,Orange,Good,Not_Checked,297,https://www.example.com/?baby=brake,http://www.example.com/?beef=acoustics
6,Orange,Bad,Checked,28,https://www.example.org/?afternoon=approval&ba...,https://www.example.com/#apparatus
7,Orange,Bad,Not_Checked,281,https://example.net/babies/badge?amount=balance,https://www.example.com/asd.php
8,Grapes,Violet,Checked,20,http://www.example.com/boundary/boat.aspx,http://basketball.example.com/bone/bedroom
9,Grapes,Violet,Not_Checked,290,http://www.example.com/sssl.php,https://www.example.org/box/back


In [163]:
df.set_index(['Labels','Pattern'])

Status  Count  \
Labels Pattern                       
Apple  Red          Checked     79   
       Red      Not_Checked    221   
       Green        Checked      3   
       Green    Not_Checked    306   
Orange Good         Checked     13   
       Good     Not_Checked    297   
       Bad          Checked     28   
       Bad      Not_Checked    281   
Grapes Violet       Checked     20   
       Violet   Not_Checked    290   

                                                        Some_Link  \
Labels Pattern                                                      
Apple  Red                                http://www.example.com/   
       Red                              http://angle.example.com/   
       Green                                 https://example.com/   
       Green                       http://www.example.com/www.php   
Orange Good                           http://example.com/blow/bag   
       Good                   https://www.example.com/?baby=brake   
       Bad      https://www.example.org/?afternoon=approval&ba...   
       Bad        https://example.net/babies/badge?amount=balance   
Grapes Violet           http://www.example.com/boundary/boat.aspx   
       Violet                     http://www.example.com/sssl.php   

                                                       Some_Link2  
Labels Pattern                                                     
Apple  Red                   http://www.example.com/beef/approval  
       Red                        https://www.example.com/qqa.php  
       Green                         https://example.com/aswq.php  
       Green    http://www.example.com/believe/bike.php?amount...  
Orange Good                              https://www.example.com/  
       Good                http://www.example.com/?beef=acoustics  
       Bad                     https://www.example.com/#apparatus  
       Bad                        https://www.example.com/asd.php  
Grapes Violet          http://basketball.example.com/bone/bedroom  
       Violet                    https://www.example.org/box/back

In [164]:
df = pd.DataFrame({'id':['x2', 'x2', 'x1', 'x1', 'x1'],
            'date':['2021-01-03','2021-01-09', '2021-01-02', '2021-01-01', '2021-01-01'],
       'distance_europe':[100, 100, 200, 200, 100],
                'distance_USA': [0, 200, 100, 100, 3]})
df

,id,date,distance_europe,distance_USA
0,x2,2021-01-03,100,0
1,x2,2021-01-09,100,200
2,x1,2021-01-02,200,100
3,x1,2021-01-01,200,100
4,x1,2021-01-01,100,3


In [165]:
df['date'] = pd.to_datetime(df['date'])

df = df.groupby([pd.Grouper(key='date', freq='W-Mon'), 'id']).sum()
df

distance_europe  distance_USA
date       id                               
2021-01-04 x1              500           203
           x2              100             0
2021-01-11 x2              100           200

In [123]:
data = [{'Year':'1959:01','0':138.89,'1':139.39,'2':139.74,'3':139.69,'4':140.68,'5':141.17},
        {'Year':'1959:07','0':141.70,'1':141.90,'2':141.01,'3':140.47,'4':140.38,'5':139.95},
        {'Year':'1960:01','0':139.98,'1':139.87,'2':139.75,'3':139.56,'4':139.61,'5':139.58}]
df = pd.DataFrame(data).set_index('Year').stack().droplevel(1)
df.index=pd.date_range(start=pd.to_datetime(df.index, format='%Y:%m')[0], 
                         periods=len(df.index), freq='M').to_period('M')                   
df = df.to_frame().reset_index().rename(columns={'index': 'Year', (0):'Value'})
df

,Year,Value
0,1959-01,138.89
1,1959-02,139.39
2,1959-03,139.74
3,1959-04,139.69
4,1959-05,140.68
5,1959-06,141.17
6,1959-07,141.70
7,1959-08,141.90
8,1959-09,141.01
9,1959-10,140.47


In [85]:
dr = pd.date_range(start='2010.1.1', periods=3, freq=3*pd.offsets.MonthBegin())

ts = pd.Series(np.random.randn(3), index=dr)
ts
ts.index
ts.asfreq(pd.offsets.MonthBegin())

2010-01-01   -1.706683
2010-02-01         NaN
2010-03-01         NaN
2010-04-01   -0.035133
2010-05-01         NaN
2010-06-01         NaN
2010-07-01    0.694031
Freq: MS, dtype: float64

In [125]:
d ="""Submit Date,Resolved Date, Count
2021-02-25,2021-03-02,1
2021-02-27,2021-03-01,2
2021-02-27,NaT,1
2021-04-01,NaT,0"""
df=g(d)
df

,Submit Date,Resolved Date,Count
0,2021-02-25,2021-03-02,1
1,2021-02-27,2021-03-01,2
2,2021-02-27,NaT,1
3,2021-04-01,NaT,0


In [166]:
df=pd.DataFrame( {
    'time': ['22:00:00', 'nan', '00:45:00', 'nan'],
    'date': ['2020-09-30T00:00:00+03:00','0001-01-01T00:00:00+02:22','2020-10-01T00:00:00+03:00','0001-01-01T00:00:00+02:22']
     })            
#df["date"]=df["date"].fillna("00/00/0000")
#df["date"] = pd.to_datetime(df["date"],errors='coerce',format='%d%m%y')
                     
#df["time"]=df["time"].fillna("00:00:00")
#df["time"] = pd.to_timedelta(df["time"].str.strip())
        
df['date'] = pd.to_datetime(df.date, errors='coerce')
#df['time'] = pd.to_datetime(df.time).dt.time.ffill()
df

df['time'] = pd.to_timedelta(df.time).ffill().astype(str).str.split().str[-1]
df

,time,date
0,22:00:00,2020-09-30 00:00:00+03:00
1,22:00:00,NaT
2,00:45:00,2020-10-01 00:00:00+03:00
3,00:45:00,NaT


In [181]:
d="""name,val
JJAT,36.0
JJEL,28.5
JJEL,36.0
JJEL,36.0
JJAA,30.0
JJEL,36.0
JJEL,22.5"""
df=pd.read_csv(StringIO(d))
df

,name,val
0,JJAT,36.0
1,JJEL,28.5
2,JJEL,36.0
3,JJEL,36.0
4,JJAA,30.0
5,JJEL,36.0
6,JJEL,22.5


In [183]:
df.groupby(['name', 'val'], as_index=False).size()
df.value_counts(['name','val']).reset_index()

,name,val,0
0,JJEL,36.0,3
1,JJAA,30.0,1
2,JJAT,36.0,1
3,JJEL,22.5,1
4,JJEL,28.5,1


In [206]:
df = pd.DataFrame({'ID': ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'D', 'D', 'E'],
                   'group': ['red', 'red', 'red', 'red', 'green', 'green', 'blue', 'blue', 'blue', 'blue', 'blue'],
                   'type': ['numer', 'denom', 'numer', 'denom', 'numer', 'denom', 'numer', 'denom', 'numer', 'denom',
                            'denom'],
                   'value': ['1', '1', '0', 'NaN', '2', '2', '1', '1', '2', '2', '2']
                   })
df

,ID,group,type,value
0,A,red,numer,1
1,A,red,denom,1
2,B,red,numer,0
3,B,red,denom,NaN
4,C,green,numer,2
5,C,green,denom,2
6,D,blue,numer,1
7,D,blue,denom,1
8,D,blue,numer,2
9,D,blue,denom,2


In [211]:
df.pivot_table('ID', ['group', 'value'], 'type', 'count').assign(val=lambda d:d['numer']/d['denom'])

type         denom  numer  val
group value                   
blue  1        1.0    1.0  1.0
      2        2.0    1.0  0.5
green 2        1.0    1.0  1.0
red   0        NaN    1.0  NaN
      1        1.0    1.0  1.0
      NaN      1.0    NaN  NaN

In [212]:
d="""Sr. No. ,Rs_Id 
1,rs17568
2,rs13303010
3,rs1921
4,rs15842
5,rs72631880"""
df=g(d)
df

,Sr. No.,Rs_Id
0,1,rs17568
1,2,rs13303010
2,3,rs1921
3,4,rs15842
4,5,rs72631880


In [213]:
d="""Sr. No.,Rs_Id,PMIDS
1,rs17568,33161796,32921275,30923998
2,rs13303010,29422604
3,rs1921,27626177,26563749
4,rs15842,26563749
5,rs72631880,."""
dfo=g(d)
dfo

,,Sr. No.,Rs_Id,PMIDS
1,rs17568,33161796,32921275.0,30923998.0
2,rs13303010,29422604,NaN,NaN
3,rs1921,27626177,26563749.0,NaN
4,rs15842,26563749,NaN,NaN
5,rs72631880,.,NaN,NaN


In [246]:
d = """Incident ID,Submit_Date,Resolved_Date
INC001,2021-02-25,2021-03-02
INC002,2021-02-27,2021-03-01
INC003,2021-02-27,
INC004,2021-04-01,"""
df=g(d)
df['Resolved_Date'] = pd.to_datetime(df.Resolved_Date)
df

,Incident ID,Submit_Date,Resolved_Date
0,INC001,2021-02-25,2021-03-02
1,INC002,2021-02-27,2021-03-01
2,INC003,2021-02-27,NaT
3,INC004,2021-04-01,NaT


In [256]:
#df['Count']=
#df.loc[:,'date1':].count(axis=1)

NaT_check = df['Resolved_Date'].isna()
df['Count'] = np.where(NaT_check, NaT_check.sum(),0)
df

,Incident ID,Submit_Date,Resolved_Date,Count
0,INC001,2021-02-25,2021-03-02,0
1,INC002,2021-02-27,2021-03-01,0
2,INC003,2021-02-27,NaT,2
3,INC004,2021-04-01,NaT,2


In [254]:
df['Resolved_Date'].isna().astype(int).sum()

2

In [263]:
d="""request_date,deal_id  
20180428,00812342
20180428,12341234
20180428,43213412
20180428,12341234
20180428,34323432
20210506,16674900"""
df=g(d)
df['request_date'] = pd.to_datetime(df.request_date, format='%Y%m%d')
df = df.set_index('request_date')
df.loc['20180428':'20180429']

,deal_id
request_date,
2018-04-28,812342
2018-04-28,12341234
2018-04-28,43213412
2018-04-28,12341234
2018-04-28,34323432


In [292]:
d="""City,Date,FeatureA,FeatureB,FeatureC,FeatureD,FeatureE,FeatureF
City A,2020-03-07,0,0,5,2,NAN,1
City B,2020-03-07,1,1,3,7,5,2
City B,2020-03-07,NAN,NAN,NAN,NAN,NAN,3
City A,2020-03-08,NaN,3,1,NAN,6,NAN"""
df=g(d)
df



,City,Date,FeatureA,FeatureB,FeatureC,FeatureD,FeatureE,FeatureF
0,City A,2020-03-07,0,0,5,2,NAN,1
1,City B,2020-03-07,1,1,3,7,5,2
2,City B,2020-03-07,NAN,NAN,NAN,NAN,NAN,3
3,City A,2020-03-08,NaN,3,1,NAN,6,NAN


In [293]:
df.Date = pd.to_datetime(df.Date)
#df = df.set_index('Date')
df


,City,Date,FeatureA,FeatureB,FeatureC,FeatureD,FeatureE,FeatureF
0,City A,2020-03-07,0,0,5,2,NAN,1
1,City B,2020-03-07,1,1,3,7,5,2
2,City B,2020-03-07,NAN,NAN,NAN,NAN,NAN,3
3,City A,2020-03-08,NaN,3,1,NAN,6,NAN


In [297]:
df.groupby('City').apply(lambda x: x.set_index('Date').resample('7D').first())

,,City,FeatureA,FeatureB,FeatureC,FeatureD,FeatureE,FeatureF
City,Date,,,,,,,
City A,2020-03-07,City A,0,0,5,2,NAN,1
City B,2020-03-07,City B,1,1,3,7,5,2


In [298]:
d="""Person_ID,Procedure_ID,Date(d/m/y)
34,30,03/03/2011
34,30,02/03/2011
32,19,01/01/2020
34,32,01/04/2012"""
df=g(d)
df

,Person_ID,Procedure_ID,Date(d/m/y)
0,34,30,03/03/2011
1,34,30,02/03/2011
2,32,19,01/01/2020
3,34,32,01/04/2012


In [299]:
df.sort_values(by='Date(d/m/y)',ascending=False).drop_duplicates(subset=['Person_ID','Procedure_ID'])

,Person_ID,Procedure_ID,Date(d/m/y)
0,34,30,03/03/2011
3,34,32,01/04/2012
2,32,19,01/01/2020


In [301]:
df.groupby(['Person_ID', 'Procedure_ID'], as_index=False).last()

,Person_ID,Procedure_ID,Date(d/m/y)
0,32,19,01/01/2020
1,34,30,02/03/2011
2,34,32,01/04/2012


In [305]:
d="""Timestamp,KE,EH,LA,PR
2013-02-27 00:00:00,1.000000,2.0 0.03,201.289993
2013-02-27 00:15:00,0.990000,2.0 0.03,210.070007
2013-02-27 00:30:00,0.950000,2.0 0.02,207.779999
2013-02-27 00:45:00,0.990000,2.0 0.03,151.960007
2013-02-27 01:00:00,341.209991,2.0 0.04,0.000000
2013-04-03 22:15:00,NaN,2.0 0.03,0.000000
2013-04-03 22:30:00,NaN,NaN 0.07,0.000000"""
df=g(d)
df

,Timestamp,KE,EH,LA,PR
0,2013-02-27 00:00:00,1.000000,2.0 0.03,201.289993,NaN
1,2013-02-27 00:15:00,0.990000,2.0 0.03,210.070007,NaN
2,2013-02-27 00:30:00,0.950000,2.0 0.02,207.779999,NaN
3,2013-02-27 00:45:00,0.990000,2.0 0.03,151.960007,NaN
4,2013-02-27 01:00:00,341.209991,2.0 0.04,0.000000,NaN
5,2013-04-03 22:15:00,NaN,2.0 0.03,0.000000,NaN
6,2013-04-03 22:30:00,NaN,NaN 0.07,0.000000,NaN


In [306]:
df.Timestamp = pd.to_datetime(df.Timestamp)

In [308]:
df.Timestamp.dt.strftime

AttributeError: 'DatetimeProperties' object has no attribute 'Hour'